In [17]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../optimized/data/AsymmWingsPointy/deployed_opt.msh')
mat = elastic_rods.RodMaterial('Rectangle', 20000, 0.3, [12,8], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

l.swapJointAngleDefinitions() # use obtuse angle instead of acute since the linkage is already opened
l.setPerSegmentRestLength(np.loadtxt('../optimized/data/AsymmWingsPointy/design_parameters.txt'))

torques = np.loadtxt('../actuation_vis/data/AsymmetricWingsPointy/torques_4.txt')
externalForces = np.zeros(l.numDoF())
externalForces[l.jointAngleDoFIndices()] = torques

jdo = l.dofOffsetForJoint(l.centralJoint())
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, 2.0, fixedVars=fixedVars)
view = LinkageViewer(l, width=1024)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [19]:
l.averageJointAngle

1.9999999999999996

In [13]:
sparseActuation = False
if (not sparseActuation): externalForces = l.gradient()

In [14]:
import write_render_files
import os
deploySeconds = 10
framesPerSecond = 30
frames = deploySeconds * framesPerSecond
outdir = 'frames_with_dense'
try: os.mkdir(outdir)
except: pass
for i, forceScale in enumerate(np.linspace(1.0, 0.0, frames)):
    framedir = "{}/{}".format(outdir, i)
    name = 'AsymmWingsPointy' if sparseActuation else 'AsymmWingsPointy_dense'
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForces, fixedVars=fixedVars)
    write_render_files.writeRenderFiles(l, framedir, name)
    if (sparseActuation): write_render_files.writeActuators(l, np.where(torques > 1e-5)[0], framedir, name)